In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Read data for X and y
df7 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/7_countbased.csv")
df15 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/15_countbased.csv")
df16 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/16_countbased.csv")
df21 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/21_countbased.csv")
df33 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/33_countbased.csv")
df56 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/56_countbased.csv")
df68 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/68_countbased.csv")
df95 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/95_countbased.csv")
df1000 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/1000_countbased.csv")
df1001 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/1001_countbased.csv")

df = pd.concat([df7,df15, df16, df21, df33, df56, df68, df95, df1000, df1001], ignore_index=True)

X = df.drop(columns=['y']).values
y = df['y'].values

In [16]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
sm = SMOTE(k_neighbors=4, random_state=42)
X_res, y_res = sm.fit_resample(X, y)

#oversampler = RandomOverSampler(random_state=42)
#X_res, y_res = oversampler.fit_resample(X,y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='manhattan')
knn.fit(X_train, y_train)

KNeighborsClassifier(metric='manhattan', weights='distance')

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Define the scoring metric
scorer = make_scorer(f1_score)

model = KNeighborsClassifier()

grid_space={'n_neighbors':[5,7,9,11,13,15],
            'metric':['euclidean','manhattan']
            }

grid = GridSearchCV(model,param_grid=grid_space,cv=2,scoring="f1_micro",verbose=10)

# Enable verbose logging using joblib
#joblib.parallel_backend('threading')

model_grid = grid.fit(X_train,y_train)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV 1/2; 1/12] START metric=euclidean, n_neighbors=5............................
[CV 1/2; 1/12] END metric=euclidean, n_neighbors=5;, score=0.481 total time=  12.8s
[CV 2/2; 1/12] START metric=euclidean, n_neighbors=5............................
[CV 2/2; 1/12] END metric=euclidean, n_neighbors=5;, score=0.499 total time=  12.9s
[CV 1/2; 2/12] START metric=euclidean, n_neighbors=7............................
[CV 1/2; 2/12] END metric=euclidean, n_neighbors=7;, score=0.509 total time=  10.4s
[CV 2/2; 2/12] START metric=euclidean, n_neighbors=7............................
[CV 2/2; 2/12] END metric=euclidean, n_neighbors=7;, score=0.505 total time=  12.2s
[CV 1/2; 3/12] START metric=euclidean, n_neighbors=9............................
[CV 1/2; 3/12] END metric=euclidean, n_neighbors=9;, score=0.512 total time=  13.5s
[CV 2/2; 3/12] START metric=euclidean, n_neighbors=9............................
[CV 2/2; 3/12] END metric=euclide

In [12]:
print('Best hyperparameters are: '+str(model_grid.best_params_))
print('Best score is: '+str(model_grid.best_score_))

Best hyperparameters are: {'metric': 'manhattan', 'n_neighbors': 15}
Best score is: 0.5150930061597716


In [8]:
y_pred = knn.predict(X_test)

KeyboardInterrupt: 

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           7       0.93      0.89      0.91     16347
          33       0.92      0.89      0.91     16457
        1000       0.83      0.90      0.86     16238

    accuracy                           0.89     49042
   macro avg       0.90      0.89      0.89     49042
weighted avg       0.90      0.89      0.89     49042



In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=knn.classes_)
disp.plot()
plt.show()

In [21]:
#Verification with new unseen data
# Read the dataframe
df_verif7 = pd.read_csv("data/Test Clean/Separated by error type/Count-based/7_countbased.csv")
df_verif33 = pd.read_csv("data/Test Clean/Separated by error type/Count-based/33_countbased.csv")
df_verif1000 = pd.read_csv("data/Test Clean/Separated by error type/Count-based/1000_countbased.csv")

# Drop any rows with missing values
df_verif = pd.concat([df_verif7, df_verif33, df_verif1000], ignore_index=True)

# Split the data into input features (X) and target variable (y)
X_verif = df_verif.drop("y", axis=1).values
y_verif = df_verif["y"].values

In [22]:
y_pred_verif = knn.predict(X_verif)
print(classification_report(y_verif, y_pred_verif))

              precision    recall  f1-score   support

           7       0.98      0.84      0.90     76652
          33       0.00      0.00      0.00        47
        1000       0.09      0.45      0.15      2272

    accuracy                           0.82     78971
   macro avg       0.36      0.43      0.35     78971
weighted avg       0.96      0.82      0.88     78971

